# Lyft Competition : Uderstanding the data (한글 번역본)

* competition: Lyft 3D Object Detection for Autonomous Vehicles
* original kernel: https://www.kaggle.com/tarunpaparaju/lyft-competition-understanding-the-data
* share: [Junghyun Park](https://github.com/parkjh688)

## Introduction
<img src="./images/image1.jpg" alt="image1" style="width: 500px;"/>

<font color='grey'>이 커널에서, 나는 이미지, LiDAR, 포인트클라우드를 포함한 데이터셋의 구성요소의 의미와 직관에 대해 설명할 것 입니다. 이러한 개념에 대한 이론을 살펴본 후, 이 데이터셋에서 정보를 더 쉽게 쿼리할 수 있는 컴팩트 형식으로 패키징할 수 있는 방법을 보여줄 것 입니다. 그리고 마지막으로, 이 데이터를 어떻게 시각화하고 탐구하는지를 matplotlib의 플롯과 그래프를 사용하여 보여줄 것 입니다.</font>

## Acknowledgements

* [NuScences DevKit ~ by Lyft](https://github.com/lyft/nuscenes-devkit)
* [EDA - 3D Object Detection Challenge ~ by beluga](https://www.kaggle.com/gaborfodor/eda-3d-object-detection-challenge)
* [Lyft: EDA, Animations, generating CSVs ~ by xhulu](https://www.kaggle.com/xhlulu/lyft-eda-animations-generating-csvs)
* [Lidar - Wikipedia](https://en.wikipedia.org/wiki/Lidar)

<font color='red'>이 커널이 흥미롭다면, 좋아요(upvote)를 눌러주세요. 좋아요는 나에게 더 많은 양질의 콘텐츠를 생산하도록 동기를 부여해줍니다 :)</font>

**자율주행 자동차를 시작해봅시다!**

이 커널의 기술적인 부분들을 시작해보기 전에, 자율주행 자동차에 대한 재미있는 비디오를 하나 봅시다!

In [1]:
from IPython.display import HTML
HTML('<center><iframe width="700" height="400" src="https://www.youtube.com/embed/tlThdr3O5Qo?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></center>')

자동차가 쉽게 교대로, 차선을 바꾸거나, 빨간 신호등에서 정지하는 등의 기능을 할 수 있다는 것을 영상에서 볼 수 있습니다. 이것은 자동차가 이미지나 LiDAR 데이터와 같은 센서의 정보를 사용하여 3D 공간에서 물체를 정확하게 인식할 수 있기 때문에 가능한 일입니다. 이제 이러한 형태의 데이터가 이론적으로 무엇을 의미하는지 살펴본 다음, 커널에서 시각화해볼 것 입니다.

## The dataset structure

1. scene - 주어진 환경에서의 25-45초의 차량 주행으로 구성됨. 각각의 scence 들은 여러 개의 샘플로 구성되어 있음.
2. sample - 특정 시간, 특정 인스턴스에 대한 스냅샷. 각 sample에는 object가 어노테이션(annotation) 되어있다.
3. sample_data - 차량의 특정 센서에서 수집한 데이터.
4. sample_annotation - 우리가 관심 있는 대상의 annotation된 인스턴스.
5. instance - 우리가 관찰한 모든 object 인스턴스의 리스트.
6. category - object 카테고리(예: 차, 사람)
7. attribute - 카테고리가 동일하게 유지되는 동안 변경될 수 있는 인스턴스의 속성.
8. visibility - 현재 사용 안함
9. sensor - 센서 종류
10. calibrated sensor - 특정 차량에서 캘리브레이션(calibration)된 특정 센서.
11. ego_pose - 특정 시간대 Ego vehicle의 상태(pose)
12. log - 데이터와 관련된 로그
13. map - 위에서 본(top-down view) binary semantic mask들

데이터의 각 스냅샷은 두 가지 형태의 정보로 구성됩니다: **이미지 데이터와 LiDAR 데이터**

이미지 데이터는 일반적입니다. .jpeg 포맷은 상당히 이해하기 쉽습니다. 각 이미지들은 단순하게 이미지를 형성하는 RGB 컬러 3개의 채널로 구성되어 있습니다:Red (R), Blue (B), and Green (G). 이 컬러 채널들을 겹쳐서 하나의 색이 있는 이미지를 만들어 냅니다. 따라서 이미지들은 4개의 차원을 가진 텐서(Tensor)로 저장되어 있습니다: (batch_size, channels, width, height).

이제, 나는 대부분의 사람들은 친숙하지 않을 LiDAR 데이터에 대해 이야기 하려고 합니다. 나는 LiDAR 데이터가 어떻게 수집되고 저장되는지에 대해 설명할 것이고, 이 데이터 포맷이 가지고 있는 의미에 대해서 이야기할 것입니다.

## What is LiDAR?

LiDAR(Light Detection and Ranging)는 주변의 정확한 3D 표현을 생성하기 위해 사용되는 방법이며, 이를 달성하기 위해 레이저 빛을 이용합니다. 기본적으로 3D 타겟은 레이저 광선(직선의 빛)으로 비춰지고 반사광은 센서에 의해 수집됩니다. 그리고 빛이 센서에 반사되는 데 필요한 시간을 계산합니다.

다른 센서들은 물체의 다른 부분으로부터 빛을 수집하며, 이 센서에 의해 기록된 시간은 다를 것입니다. 센서가 계산한 시간의 차는 물체의 깊이를 계산하는 데 사용될 수 있습니다. 이 깊이 정보는 이미지의 2D 표현과 결합되어 객체의 정확한 3D 표현을 제공하게 됩니다. 이 과정은 실제 인간의 시각과 비슷합니다. 우리의 두 눈은 2D로 관찰하고, 이 두 개의 정보를 결합해서 3D 지도(depth perception)를 형성합니다. 이것이 바로 인간이 우리 주변의 세상을 이렇게 감지하는 방법입니다.

이 기술은, 많은 실제 시나리오에서 3D 표현을 만드는 데 사용됩니다. 예를 들어, 농장에서 씨앗을 뿌리고 잡초를 제거하는 것을 돕기 위해 사용됩니다. 움직이는 로봇은 LiDAR를 사용하여 주변 환경에 대한 3D 지도를 만들고 이 지도를 사용하여 장애물을 피하고 임무를 완수합니다. 이 기술은 고고학에서도 사용됩니다. LiDAR는 2D 유물(artifact) 스캔본의 3D 렌더링하는데 사용됩니다. 이것은 어떤 방법으로도 유물을 발굴할 수 없을 때 유물의 3차원 형상을 정확히 알 수 있게 해줍니다. 마지막으로, LiDAR는 또한 해저와 접근이 불가능한 다른 지형들의 고품질 3D 지도를 만드는 데 사용될 수 있어 지질학자들과 해양학자들에게 매우 유용합니다. 아래는 LiDAR을 사용하여 생성된 해저의 3D 지도입니다.

<img src="./images/image2.jpg" alt="image2" style="width: 500px;"/>

물론, 자율주행차는 이 기술을 사용하여 주변의 물체를 3D로 식별하고, 이 물체의 속도와 방향을 추정합니다. 이 포괄적인 3D 지도는 복잡한 환경에서도 차량을 탐색할 수 있도록 상세한 정보를 제공합니다. 아래는 LiDAR를 장착한 드론이 나오는 영상입니다. 위에서 언급한 과정을 이용하여 자동적으로 그 주위에 3D 지도를 만듭니다.

In [9]:
HTML('<center><iframe width="700" height="400" src="https://www.youtube.com/embed/x7De3tCb3_A?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></center>')

<br></br>
<br></br>
## How does LiDAR work?

<img src="./images/image3.gif" alt="image3" style="width: 300px;"/>

위의 GIF는 LiDAR가 어떻게 작동하는지 대략적으로 보여주고 있습니다. 기본적으로, 레이저 광선은 레이저에 의해 사방으로 발사됩니다. 레이저 빔은 경로에 있는 물체를 반사하고 반사된 빛은 센서에 의해 수집됩니다. 현재 Flash LiDAR 카메라라고 불리는 특별한 장치가 이러한 센서의 정보를 사용하여 3D 지도를 만드는 데 사용되고 있습니다.

### Flash LiDAR Camera

<img src="./images/image4.jpg" alt="image4" style="width: 200px;"/>

위의 이미지에 나타난 장치를 Flash LiDAR 카메라라고 합니다. Flash LiDAR 카메라의 초점면에는 3D 모델을 만들 수 있는 "depth(깊이)"와 "intensity(강도)" 픽셀의 행과 열이 있습니다. 각각의 픽셀은 레이저 펄스가 목표물에 부딪히고 센서로 돌아오는 데 걸리는 시간과 레이저 펄스에 의해 접촉되는 물체의 깊이, 위치, 반사 강도를 기록합니다.

Flash LiDAR 단일 펄스로 시야를 밝히는 단일 광원을 사용합니다. 컬러 대신 거리 사진을 찍는 카메라처럼.

온보드 조명 소스(onboard source of illumination)는 Flash LiDAR를 활성 센서로 만듭니다. 반환되는 신호는 내장된 알고리즘에 의해 처리되어 센서의 시야 내에서 물체와 지형 형상의 거의 즉각적인 3D 렌더링을 생성하게 됩니다. 레이저 펄스 반복 주파수는 높은 해상도와 정확도를 가진 3D 비디오를 생성하기에 충분합니다. 센서의 프레임률이 높기 때문에 자율주행 등 실시간 시각화가 필요한 다양한 어플리케이션에 유용한 툴입니다. 대상 환경의 3D 고도 메쉬를 즉시 반환함으로써, 무인자동차가 속도 조정, 제동, 조향 등과 관련된 결정을 내릴 수 있게 합니다.

이런 형태의 카메라는 자율주행차 상단에 부착되어 있으며, 이 차들은 주행 중 이를 이용해 움직입니다.

<br></br>
<br></br>
이제 LiDAR가 무엇인지, 어떻게 작동하는지 명확하게 알게되었으므로 데이터셋을 바로 시각화해볼 수 있습니다.

<br></br>
<br></br>
## Visualizing the data¶

### lyft_dataset_sdk 및 필요한 라이브러리 설치 하기

<br></br>
우리는 이미지와 LiDAR 데이터를 쉽게 시각화하기 위해 lyft_dataset_sdk 라이브러리가 필요합니다. 간단한 pip install 명령만 있으면 설치는 끝납니다. 또한 인터랙티브(interactive) 플롯을 생성하기 위해 chart_studio 라이브러리를 설치할 것입니다.

In [4]:
!pip install lyft_dataset_sdk

  Using cached scikit_learn-0.22.1-cp37-cp37m-manylinux1_x86_64.whl (7.0 MB)
     |████████████████████████████████| 235 kB 755 kB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 33.8 MB/s eta 0:00:01
     |████████████████████████████████| 28.2 MB 152 kB/s eta 0:00:011
     |████████████████████████████████| 7.1 MB 104.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 37.5 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 1.4 MB/s  eta 0:00:01
  Using cached cachetools-4.0.0-py3-none-any.whl (10 kB)
     |████████████████████████████████| 13.1 MB 933 kB/s eta 0:00:011
     |████████████████████████████████| 10.0 MB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 1.7 MB/s  eta 0:00:01
  Using cached scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl (26.1 MB)
     |████████████████████████████████| 76 kB 859 kB/s  eta 0:00:01
     |████████████████████████████████| 20.1 MB 1.3 MB/s eta 0:00:011
     |██████████████████████████████

이제 데이터 탐구를 하는데 필요한 다른 라이브러리도 가져와 보겠습니다.

In [2]:
import os
import gc
import numpy as np
import pandas as pd

import json
import math
import sys
import time
from datetime import datetime
from typing import Tuple, List

import cv2
import matplotlib.pyplot as plt
import sklearn.metrics
from PIL import Image

from matplotlib.axes import Axes
from matplotlib import animation, rc
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import plot, init_notebook_mode
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

import seaborn as sns
from pyquaternion import Quaternion
from tqdm import tqdm

from lyft_dataset_sdk.utils.map_mask import MapMask
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.geometry_utils import view_points, box_in_image, BoxVisibility
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from pathlib import Path

import struct
from abc import ABC, abstractmethod
from functools import reduce
from typing import Tuple, List, Dict
import copy

<br></br>
<br></br>
**데이터셋 경로(path) 맞추기**

In [3]:
DATA_PATH = '../input/3d-object-detection-for-autonomous-vehicles/'

**학습 데이터가 저장되어 있는 데이터 프레임 가져오기**

In [6]:
train = pd.read_csv(DATA_PATH + 'train.csv')
sample_submission = pd.read_csv(DATA_PATH + 'sample_submission.csv')

**카테고리별로 데이터 그룹짓기**

In [7]:
# Taken from https://www.kaggle.com/gaborfodor/eda-3d-object-detection-challenge

object_columns = ['sample_id', 'object_id', 'center_x', 'center_y', 'center_z',
                  'width', 'length', 'height', 'yaw', 'class_name']
objects = []
for sample_id, ps in tqdm(train.values[:]):
    object_params = ps.split()
    n_objects = len(object_params)
    for i in range(n_objects // 8):
        x, y, z, w, l, h, yaw, c = tuple(object_params[i * 8: (i + 1) * 8])
        objects.append([sample_id, i, x, y, z, w, l, h, yaw, c])
train_objects = pd.DataFrame(
    objects,
    columns = object_columns
)

100%|██████████| 22680/22680 [00:01<00:00, 19501.70it/s]


**숫자인 피쳐(features) str에서 float32로 바꾸기**

In [8]:
numerical_cols = ['object_id', 'center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'yaw']
train_objects[numerical_cols] = np.float32(train_objects[numerical_cols].values)

In [9]:
train_objects.head()

,sample_id,object_id,center_x,center_y,center_z,width,length,height,yaw,class_name
0,db8b47bd4ebdf3b3fb21598bb41bd8853d12f8d2ef25ce...,0.0,2680.282959,698.196899,-18.047768,2.064,5.488,2.053,2.604164,car
1,db8b47bd4ebdf3b3fb21598bb41bd8853d12f8d2ef25ce...,1.0,2691.997559,660.801636,-18.674259,1.818,4.570,1.608,-0.335176,car
2,db8b47bd4ebdf3b3fb21598bb41bd8853d12f8d2ef25ce...,2.0,2713.607422,694.403503,-18.589972,1.779,4.992,1.620,2.579456,car
3,db8b47bd4ebdf3b3fb21598bb41bd8853d12f8d2ef25ce...,3.0,2679.986816,706.910156,-18.349594,1.798,3.903,1.722,2.586166,car
4,db8b47bd4ebdf3b3fb21598bb41bd8853d12f8d2ef25ce...,4.0,2659.352051,719.417480,-18.442999,1.936,4.427,1.921,2.601799,car


<br></br>
<br></br>
**첫 번째 데이터 탐구**

이제, 나는 이 특정 데이터 프레임의 데이터를 살펴보고 그것으로부터 유용한 인사이트를 얻을 수 있는지 알아볼 것입니다.

**center_x와 center_y**

center_x와 center_y는 객체 위치의 중심(bounding volume)의 x, y 좌표에 해당합니다. 이 좌표는 x-y평면에 있는 객체의 위치를 나타냅니다.

<br></br>
**center_x와 center_y의 분포**

<img src="./images/image5.png" alt="image5" style="width:600px;"/>

위의 다이어그램에서, 보라색은 center_y의 분포를, 주황색은 center_x의 분포를 나타냅니다. 위의 다이어그램에서 알 수 있듯이, 우리는 center_x, center_y 모두 여러개의 피크를 가지며 이것으로부터 멀티모달(multimodal)임을 알 수 있습니다. 두 분포 모두 right skewed되어 있습니다. 하지만 center_y의 분포(보라색)은 center_x(주황색)의 분포보다 훨씬 더 치우쳐져(skewed) 있습니다. center_x 분포가 더 고르게 분포되어 있기 때문에 알 수 있었습니다.

이것은 물체가 x축을 따라 매우 고르게 펼쳐지지만 y축은 아니라는 것을 나타냅니다. 도로 폭이 작아 자동차의 카메라가 왼쪽이나 오른쪽의 물체를 쉽게 감지할 수 있기 때문일 것입니다. 그러나 도로의 길이가 폭보다 훨씬 크고, 카메라의 시야가 이 각도에서 차단될 가능성이 높기 때문에 카메라는 앞이나 뒤쪽으로 좁게 물체를 찾을 수 있었을 것입니다.

<br></br>
<br></br>
**center_x와 center_y의 관계**

**KDE Plot**

<img src="./images/image6.png" alt="image6" style="width:400px;"/>

위의 KDE 그림에서, center_x와 center_y는 다소 부정적인 상관관계를 가지고 있는 것 같이 보입니다. 이것은 아마도, 다시 한 번, 카메라 시스템의 한계 때문에 일일 것이라고 생각됩니다. 카메라의 앞쪽에 있으면서 옆으로는 가까운 물체를 감지할 수 있습니다. 그리고, 그것은 또한 멀지 않은 물체를 감지할 수 있으면서, 옆으로 멀리 있는 물체는 감지할 수 없습니다. **그러나 카메라는 앞쪽으로 멀리있으면서 옆으로도 멀리 있는 물체를 감지할 수 없습니다.** 그렇게 때문에 훨씬 앞쪽에 있고 옆으로도 멀리 떨어져 있는 물체는 전혀 검출되지 않고, 그 조건 중 하나(또는 둘 다 아닐 때)를 만족시키는 물체만 검출합니다. 이것은 center_x와 center_y 사이에 부정적인 관계를 만듭니다.

<br></br>
**center_z**

center_z는 객체 위치의 중심(bounding volume)의 xz좌표에 해당한다. 이 좌표는 x-y 평면 위에 있는 물체의 높이를 나타냅니다.

**center_z의 분포**

<img src="./images/image7.png" alt="image7" style="width:400px;"/>

위 다이어그램에서, 우리는 center_z의 분포가 극히 높은 양의(우측) 치우침을 가지고 있으며 -20 마크(평균값에 근접한 값)를 중심으로 군집되어 있음을 알 수 있습니다. center_z의 분산(스프레드)은 center_x와 center_y보다 상당히 작다는 것을 알 수 있습니다. 이것은 아마도 대부분의 물체가 도로의 평평한 평면에 매우 가깝기 때문에 카메라 위(또는 아래)의 물체의 높이에는 큰 변화가 없기 때문일 것입니다. 개체의 x와 y의 좌표는  큰 분산이 있습니다.

또 카메라가 무인 자동차 위에 부착돼 있어 대부분의 z 좌표는 음수입니다. 그래서, 대부분의 경우 카메라는 물체를 보기 위해 "아래를 내려다봐야" 합니다. 따라서 카메라와 상대적인 물체의 높이 혹은 z 좌표가 일반적으로 음수인 것 입니다.

<br></br>
**yaw**

**yaw**는 z축을 중심으로 한 각도로, 지상에 있을 때 차량/경계 박스의 앞부분이 가리키는 방향을 'yaw'로 만듭니다.

**yaw의 분포**

<img src="./images/image8.png" alt="image8" style="width:500px;"/>

위 다이어그램에서, 우리는 yaw의 분포가 대략 bimodal, 즉 분포에 2개의의 큰 피크(peaK)가 있음을 알 수 있습니다. 피크 중 하나는 약 0.5, 다른 하나는 약 2.5점 정도로 보입니다. mean은 1과 2(약 1.5) 사이라고 추정할 수 있습니다. 분포에는 뚜렷한 치우침(skew)이 없습니다. 대칭 위치에 있는 두 개의 피크는 치우침(skew)를 감소시키고, center_x, center_y, center_z의 분포보다 분포가 균형이 있게 해줍니다.

<br></br>
**width**

width는 간단히 놓여있는 물체의 bounding volume의 폭입니다.

<img src="./images/image9.png" alt="image9" style="width:500px;"/>

위의 다이어그램에서, width는 대략 보통 약 2를 평균으로 하면서 양쪽에 일부 특이치가 있는 것으로 볼 수 있습니다. 대다수의 물체는(나중에 보게 되겠지만) 자동차로, 약 2 정도의 width을 이룹니다. 오른쪽의 특이치는 트럭이나 밴과 같은 큰 객체들을 나타내고, 왼쪽의 특이치는 보행자나 자전거와 같은 작은 물체를 나타냅니다.

<br></br>
**length**

length는 간단히 놓여있는 물체의 bounding volume의 길이입니다.

<img src="./images/image10.png" alt="image10" style="width:500px;"/>

위의 다이어그램에서, length는 강한 positive skew(rightward skew)를 가지고 있으며, 평균은 약 5이고, 어느 한쪽에는 약간의 특이치가 있다는 것을 알 수 있습니다. 대다수의 물체는(나중에 보게 되겠지만) 자동차로, 이것들은 길이 약 5를 이룹니다. 오른쪽의 특이치는 트럭이나 밴과 같은 큰 오브젝트들을 나타내고, 왼쪽의 특이치는 보행자나 자전거와 같은 작은 물체를 나타냅니다.

<br></br>
**height**

height는 단순히 놓여 있는 경계 부피의 높이입니다.

<img src="./images/image11.png" alt="image11" style="width:500px;"/>

위의 다이어그램에서, height는 평균이 약 2이면서 positive skew(rightward skew)를 가지고 있으며, 어느 한쪽에는 일부 특이치가 있음을 알 수 있습니다. 대다수의 물체는(나중에 보게 되겠지만) 자동차로, 이것들은 (정점에서) 약 2의 길이를 이룬다. 오른쪽의 특이치는 트럭이나 밴과 같은 큰 오브첵트들을 나타내고, 왼쪽의 특이치는 보행자나 자전거와 같은 작은 물체를 나타냅니다.

<br></br>
**카테고리별 객체의 빈도**

<img src="./images/image12.png" alt="image12" style="width:600px;"/>

위의 다이어그램에서, 데이터셋의 가장 일반적인 객체 클래스는 "car"라고 볼 수 있습니다. 캘리포니아 실리콘밸리의 팔로 알토 거리에서 찍은 사진이기 때문에 이것은 놀라운 일이 아닙니다. 그리고, 도로들에서 가장 일반적인 차량은 자동차들입니다. 다른 모든 물체들은 빈도 수로는 자동차 근처에도 못 따라 간다는 것을 알 수 있습니다.

<br></br>
**center_x** <i>**vs.**</i> **class_name**

아래 그림에서, center_x 분포가 다른 객체들의 class_names에 대해 어떻게 변하는지 살펴보겠습니다.

<img src="./images/image13.png" alt="image13" style="width:600px;"/>

위의 바이올린 플롯에서, 트럭, 버스, 기타 차량을 포함한 대형 차량에 대한 center_x의 분포가 잘 퍼져 있음을 알 수 있습니다. 그들은 거의 비뚤어진 데가 없고 보행자와 자전거의 분포보다 더 잘 퍼져있습니다. 이는 아마도 이런 대형 차량들이 다른 차들과 더 먼 거리를 유지하는 경향이 있고, 소형차량들은 사고를 피하기 위해 이런 대형차들과 너무 가까이 있지 않기 때문일 것입니다. 따라서 평균 center_x는 버스나 트럭과 같은 대형 차량에 대해 분명히 더 큽니다.

이와는 대조적으로, 보행자나 자전거와 같은 작은 물체는 cneter_x 분포에 강한 positive skew(rightward)를 가지고 있습니다. 이러한 분포는 또한 대형 차량에 대한 분배보다 확실히 작은 평균을 가지고 있습니다. 보행자와 자전거 이용자들이 사고를 피하기 위해 자동차나 트럭으로 큰 거리를 유지할 필요가 없기 때문일 것입니다. 보통 빨간불이 켜지고, 차들이 멈췄을 때 도로를 건너기 때문입니다.

<img src="./images/image14.png" alt="image14" style="width:600px;"/>

위의 박스 플롯에서, 바이올린 플롯에서 얻은 정보들과 같다는 것을 눈치챌 수 있습니다. 보행자나 자전거와 같은 작은 물체에 대한 중심_x 분포는 자동차, 트럭, 버스 같은 큰 물체에 비해 매우 낮은 평균값과 사분위수 값을 가진다는 것을 알 수 있습니다.

<br></br>
**center_y** <i>**vs.**</i> **class_name**

아래 플롯에서, center_y의 분포가 class_names(다른 객체)에 어떻게 변하는지 탐구할 것 입니다.

<img src="./images/image15.png" alt="image15" style="width:600px;"/>

위의 바이올린 플롯에서, 우리는 보행자와 자전거를 포함한 작은 물체에 대한 center_y의 분포는 트럭이나 버스 같은 큰 물체보다 더 큰 평균값을 가지고 있음을 알 수 있습니다. 작은 물체에 대한 분포는 큰 물체에 비해 center_y의 높은 값에 집중된 확률 밀도가 훨씬 큽니다. 이는 일반적으로 작은 물체가 큰 물체보다 center_y 값이 크다는 것을 의미합니다.